In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 32
tree_depth = 6
batch_size = 512
device = 'cpu'
train_data_path = r'/mnt/qnap/ekosman/mitbih_train.csv'
test_data_path = r'/mnt/qnap/ekosman/mitbih_test.csv'

In [3]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [4]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [5]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [6]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [ ]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 4.4404826164245605 | KNN Loss: 3.9451022148132324 | CLS Loss: 0.4953805208206177
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 4.513814926147461 | KNN Loss: 3.9397802352905273 | CLS Loss: 0.5740345120429993
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 4.455218315124512 | KNN Loss: 3.922706365585327 | CLS Loss: 0.5325121283531189
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 4.294148921966553 | KNN Loss: 3.8838322162628174 | CLS Loss: 0.4103168845176697
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 4.294412136077881 | KNN Loss: 3.7883591651916504 | CLS Loss: 0.5060530304908752
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 4.268202304840088 | KNN Loss: 3.8272018432617188 | CLS Loss: 0.4410002529621124
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 4.186133861541748 | KNN Loss: 3.787473440170288 | CLS Loss: 0.39866048097610474
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 4.192091941833496 | KNN Loss: 3.803377866744995 |

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 3.7460460662841797 | KNN Loss: 3.6439363956451416 | CLS Loss: 0.1021096259355545
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 3.8521170616149902 | KNN Loss: 3.677859306335449 | CLS Loss: 0.17425784468650818
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 3.7913012504577637 | KNN Loss: 3.6585447788238525 | CLS Loss: 0.13275651633739471
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 3.7754223346710205 | KNN Loss: 3.682349681854248 | CLS Loss: 0.09307257831096649
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 3.7439205646514893 | KNN Loss: 3.5736303329467773 | CLS Loss: 0.17029017210006714
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 3.753046751022339 | KNN Loss: 3.6106090545654297 | CLS Loss: 0.14243777096271515
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 3.7714054584503174 | KNN Loss: 3.5768871307373047 | CLS Loss: 0.1945182979106903
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 3.8094074726104736 | KNN Loss: 3.

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 3.6876981258392334 | KNN Loss: 3.6275582313537598 | CLS Loss: 0.06013982743024826
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 3.697531223297119 | KNN Loss: 3.6259424686431885 | CLS Loss: 0.07158877700567245
Epoch: 007, Loss: 3.7277, Train: 0.9772, Valid: 0.9739, Best: 0.9739
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 3.6972131729125977 | KNN Loss: 3.6047801971435547 | CLS Loss: 0.0924328863620758
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 3.7043237686157227 | KNN Loss: 3.6328940391540527 | CLS Loss: 0.07142966985702515
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 3.691478967666626 | KNN Loss: 3.6424553394317627 | CLS Loss: 0.049023743718862534
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 3.7200684547424316 | KNN Loss: 3.6394450664520264 | CLS Loss: 0.08062346279621124
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 3.663825273513794 | KNN Loss: 3.589221715927124 | CLS Loss: 0.07460358738899231
Epoch 8 / 200 | i

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 3.7020018100738525 | KNN Loss: 3.611156940460205 | CLS Loss: 0.09084483981132507
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 3.6834349632263184 | KNN Loss: 3.5963950157165527 | CLS Loss: 0.08704005926847458
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 3.6913721561431885 | KNN Loss: 3.604405164718628 | CLS Loss: 0.08696708083152771
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 3.6948282718658447 | KNN Loss: 3.6119637489318848 | CLS Loss: 0.08286451548337936
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 3.737065315246582 | KNN Loss: 3.6354610919952393 | CLS Loss: 0.10160432755947113
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 3.7306935787200928 | KNN Loss: 3.6641175746917725 | CLS Loss: 0.0665760412812233
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 3.6791985034942627 | KNN Loss: 3.5801010131835938 | CLS Loss: 0.09909740090370178
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 3.7615909576416016 | KNN Lo

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 3.7090442180633545 | KNN Loss: 3.6246423721313477 | CLS Loss: 0.0844019427895546
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 3.6484458446502686 | KNN Loss: 3.5808098316192627 | CLS Loss: 0.06763608008623123
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 3.7904977798461914 | KNN Loss: 3.726064443588257 | CLS Loss: 0.06443341821432114
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 3.6699464321136475 | KNN Loss: 3.6021406650543213 | CLS Loss: 0.06780566275119781
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 3.6888301372528076 | KNN Loss: 3.6273884773254395 | CLS Loss: 0.061441775411367416
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 3.653918743133545 | KNN Loss: 3.6003856658935547 | CLS Loss: 0.05353296548128128
Epoch: 014, Loss: 3.6835, Train: 0.9857, Valid: 0.9805, Best: 0.9805
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 3.6623129844665527 | KNN Loss: 3.624149799346924 | CLS Loss: 0.03816324472427368
Epoch

Epoch 18 / 200 | iteration 10 / 171 | Total Loss: 3.686664342880249 | KNN Loss: 3.6441001892089844 | CLS Loss: 0.042564064264297485
Epoch 18 / 200 | iteration 20 / 171 | Total Loss: 3.703777313232422 | KNN Loss: 3.6534957885742188 | CLS Loss: 0.050281405448913574
Epoch 18 / 200 | iteration 30 / 171 | Total Loss: 3.650564193725586 | KNN Loss: 3.5808115005493164 | CLS Loss: 0.06975264847278595
Epoch 18 / 200 | iteration 40 / 171 | Total Loss: 3.637263774871826 | KNN Loss: 3.5902106761932373 | CLS Loss: 0.04705320671200752
Epoch 18 / 200 | iteration 50 / 171 | Total Loss: 3.6588711738586426 | KNN Loss: 3.620802640914917 | CLS Loss: 0.03806859627366066
Epoch 18 / 200 | iteration 60 / 171 | Total Loss: 3.666327714920044 | KNN Loss: 3.6336326599121094 | CLS Loss: 0.0326949767768383
Epoch 18 / 200 | iteration 70 / 171 | Total Loss: 3.6621155738830566 | KNN Loss: 3.610682249069214 | CLS Loss: 0.05143332481384277
Epoch 18 / 200 | iteration 80 / 171 | Total Loss: 3.6864466667175293 | KNN Loss: 3

Epoch 21 / 200 | iteration 80 / 171 | Total Loss: 3.6463847160339355 | KNN Loss: 3.587380886077881 | CLS Loss: 0.0590038038790226
Epoch 21 / 200 | iteration 90 / 171 | Total Loss: 3.713458299636841 | KNN Loss: 3.5985424518585205 | CLS Loss: 0.11491577327251434
Epoch 21 / 200 | iteration 100 / 171 | Total Loss: 3.639531135559082 | KNN Loss: 3.600400924682617 | CLS Loss: 0.03913012519478798
Epoch 21 / 200 | iteration 110 / 171 | Total Loss: 3.652320623397827 | KNN Loss: 3.596884250640869 | CLS Loss: 0.05543634667992592
Epoch 21 / 200 | iteration 120 / 171 | Total Loss: 3.642812967300415 | KNN Loss: 3.5828213691711426 | CLS Loss: 0.05999155342578888
Epoch 21 / 200 | iteration 130 / 171 | Total Loss: 3.6397979259490967 | KNN Loss: 3.6042985916137695 | CLS Loss: 0.03549940511584282
Epoch 21 / 200 | iteration 140 / 171 | Total Loss: 3.6662955284118652 | KNN Loss: 3.62724232673645 | CLS Loss: 0.03905318304896355
Epoch 21 / 200 | iteration 150 / 171 | Total Loss: 3.619785785675049 | KNN Loss: 

In [ ]:
test(model, test_data_iter, device)

In [ ]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

In [ ]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

In [ ]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

In [ ]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [ ]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

In [ ]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [ ]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [ ]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [ ]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [ ]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [ ]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [ ]:
losses = []
accs = []
sparsity = []

In [ ]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

In [ ]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

In [ ]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

# Tree Visualization

In [ ]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

# Extract Rules

# Accumulate samples in the leaves

In [ ]:
print(f"Number of patterns: {len(root.get_leaves())}")

In [ ]:
method = 'greedy'

In [ ]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

# Tighten boundaries

In [ ]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")